In [17]:
from sympy import *
import numpy as np
import scipy as sp
from scipy.linalg import lu
from scipy.sparse.linalg import spilu
import math
import sympy 
import itertools
import sys
import time
import multiprocessing

from __future__ import print_function


from copy import deepcopy
import time
import fractions
import timeit
from fractions import Fraction

import matplotlib.pyplot as plt

%matplotlib inline

%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# print function

In [18]:
def print_row(row,symbols,nvar,precision,cal=False,tol = 10**-5 ):
    # nvar  
    plus_flag = False
    
    for ele in row:
        mul_flag = False
        if ele[0] == 0:
            continue
        constant_flag = True
        for i in range(nvar+1):
            if i==0:
                if plus_flag and ele[i]>0:
                    print('+',end ='')
                if ele[i]==-1.:
                    print('-',end ='')
                    continue
                if ele[i] != 1.:
                    if cal or precision:
                        if cal:
                            mul_flag = True
                        print(ele[i],end="")
                    else:
                        print("%2.2f"%ele[i],end="")
                plus_flag = True
                continue
            if ele[i] != 0:
                if mul_flag:
                    print('*',end = "")
                if cal :
                    mul_flag = True
                print(symbols[i-1],end = "")
                if ele[i] > 1:
                    if cal:
                        print("**",end="")
                    else:
                        print("^",end="")
                    print(int(ele[i]),end="")
                constant_flag = False
        
        if constant_flag and ele[0]==1 :
            if cal or precision:
                print(ele[0],end="")
            else:
                print("%2.2f"%ele[0],end="")
        if constant_flag and ele[0]==-1 :
            if cal or precision:
                print(1,end="")
            else:
                print("%2.2f"%1,end="")
        plus_flag = True
    if cal:
        print(",")
    else:
        print("\\\\")
            
def print_all(M,nvar,symbols=[],cal=False,precision = False):
    if len(symbols) == 0:
        symbols = ['x0','x1','x2','x3','x4','x5','x6','x7'][:nvar]
    for row in M:
        if len(row)!=0:
            print_row(row,symbols,nvar,precision,cal)
    print("")

## Generate Gauissian Quaduature 

In [52]:
# nomal Gaussian Quaduature
def GQ(n=2,Float = False):
    result = []
    for i in range(2*n):
        p = []
        for j in range(n):
            p_s = np.zeros(2*n+1,dtype = int)
            p_s[0] = 1
            p_s[2*j+1] = 1
            p_s[2*j+2] = i
            p.append(p_s.tolist())
        if i-i/2*2 == 0 and Float:
            p_s = np.zeros(2*n+1)
            p_s[0] = -2./(i+1)
            p.append(p_s.tolist())
        elif i-i/2*2 == 0:
            p_s = np.zeros(2*n+1,dtype = object)
            p_s[0] = Fraction(-2,i+1)
            p.append(p_s.tolist())
            
        result.append(p)
    return result

def GQ_sym(n=2,Latex = False):        
    result = []
    
    if Latex:
        for i in range(2*n):
            if i-i/2*2 == 0:
                c = "w_%d"%(i/2)
                result.append(c)
            else:
                c = "x_%d"%(i/2)
                result.append(c)
        return result
    
    for i in range(2*n):
        if i-i/2*2 == 0:
            c = "w%d"%(i/2)
            result.append(c)
        else:
            c = "x%d"%(i/2)
            result.append(c)
    return result

In [20]:
def GQS(n=2):
    odd = False
    if n-n/2*2 == 1:
        odd = True
    result = []
    if True:
        for i in range(2*n):
            p = []
            for j in range(n/2):
                p_s = np.zeros(2*n - n/2*2+1,dtype = int)
                p_s[0] = 1
                p_s[2*j+1] = 1
                p_s[2*j+2] = i
                p.append(p_s.tolist())
                
                p_s = np.zeros(2*n - n/2*2+1,dtype = int)
                p_s[0] = 1
                if i-i/2*2 >0:
                    p_s[0] = -1
                p_s[2*j+1] = 1
                p_s[2*j+2] = i
                p.append(p_s.tolist())
            if odd:
                p_s = np.zeros(2*n - n/2*2+1,dtype = int)
                p_s[0] = 1
                p_s[2*n - n/2*2-1] = 1
                p_s[2*n - n/2*2] = i
                p.append(p_s.tolist())
            
            if i-i/2*2 == 0 :
                p_s = np.zeros(2*n - n/2*2+1)
                p_s[0] = -2./(i+1)
                p.append(p_s.tolist())
            
            result.append(p)
        return result
    for i in range(2*n):
        p = []
        for j in range(n/2):
            p_s = np.zeros(2*n+1,dtype = int)
            p_s[0] = 1
            p_s[2*j+1] = 1
            p_s[2*j+2] = i             
            p.append(p_s.tolist())
                
            p_s = np.zeros(2*n+1,dtype = int)
            p_s[0] = 1
            if i-i/2*2 >0:
                p_s[0] = -1
            p_s[2*j+1] = 1
            p_s[2*j+2] = i
            p.append(p_s.tolist())
        if i-i/2*2 == 0:
            p_s = np.zeros(n+1)
            p_s[0] = -2./(i+1)
            p.append(p_s.tolist())
        result.append(p)
    return result


# formation of the coefficient matri for multi-variable

In [22]:
class CSR:
    
    def __init__(self, arg1,shape):
        if len(arg1) == 3:
            self.data = np.array(arg1[0])
            self.indices = np.array(arg1[1])
            self.indptr = np.array(arg1[2])
            self.shape = shape
            self.nnz = len(self.data)
            self.eliminate_zeros()
            
            if len(self.data) != len(self.indices):
                print("indices does not match")
            if len(self.data) != self.indptr[-1]:
                print("indptr does not match")
            
            
        if len(arg1) == 2:
            self.data =[]
            self.indices = []
            self.indptr = []
            row_data = arg1[0]
            ind = arg1[1]
            self.shape = shape
            
            n = np.array(row_data).shape[0]
            if (np.array(ind[0]).shape[0] != n):
                print("Row index does not match")
            if (np.array(ind[1]).shape[0] != n):
                print("Column index does not match")

            # Sort
            for j in range(n):
                for i in range(j,n):
                    if ind[0][i] < ind[0][0]:
                        a = deepcopy(row_data[0])
                        b = deepcopy(ind[0][0])
                        c = deepcopy(ind[1][0])
                        
                        ind[0][0] = deepcopy(ind[0][i])
                        ind[1][0] = deepcopy(ind[1][i])
                        row_data[0] = row_data[i]
                        
                        row_data[i] = a
                        ind[0][i] = b
                        ind[1][i] = c
                    
                    else:
                        if ind[0][i] == ind[0][0] and ind[1][i]<ind[1][0]:
                            a = deepcopy(row_data[0])
                            b = deepcopy(ind[0][0])
                            c = deepcopy(ind[1][0])

                            ind[0][0] = deepcopy(ind[0][i])
                            ind[1][0] = deepcopy(ind[1][i])
                            row_data[0] = row_data[i]

                            row_data[i] = a
                            ind[0][i] = b
                            ind[1][i] = c
            
            self.data = []
            ind_new = [[],[]]      
            
            # item in same position
            if np.array(row_data).shape[0] == 0:
                self.indices = []
                self.indptr = []
                for j in range(shape[0]+1):
                    self.indptr.append(0)
                return
            i = 1
            row_no = ind[0][0]
            col_no = ind[1][0]
            same_no = 1
            while i < np.array(row_data).shape[0]:    
                if ind[0][i] == row_no and ind[1][i] == col_no:
                    same_no += 1
                else:
                    s = row_data[i-1]
                    for j in range(same_no-1):
                        s += row_data[i - j - 2]
                    self.data.append(s)
                    ind_new[0].append(row_no)
                    ind_new[1].append(col_no)
                    same_no = 1
                    row_no = ind[0][i]
                    col_no = ind[1][i]
                i += 1
            
            s = row_data[i-1]
            for j in range(same_no-1):
                s += row_data[i - j - 2]
            self.data.append(s)
            ind_new[0].append(row_no)
            ind_new[1].append(col_no)
                    
            
            for e in ind_new[1]:
                self.indices.append(int(e))
            self.indptr.append(0)
            
            current_row = 0
            for i in range(np.array(self.data).shape[0]):
                if ind_new[0][i] != current_row:
                    for j in range(ind_new[0][i]-current_row):
                        self.indptr.append(i)
                    current_row = ind_new[0][i]
            self.indptr.append(np.array(self.data).shape[0]) 
            
            self.indptr = np.array(self.indptr)
            self.indices = np.array(self.indices)
            self.data = np.array(self.data)
            self.nnz = len(self.data)
            
            for row in range(self.shape[0]):
                array_sort(self.data,self.indices,self.indptr[row],self.indptr[row+1])
            
    
    def array_sort(data,indices,start,end):
        for i in range(start,end):
            for j in range(i,end):
                if indices[j] < indices[i]:
                    a = indices[j]
                    indices[j] = indices[i]
                    indices[i] = a
                    
                    b = data[j]
                    data[j] = data[i]
                    data[i] = b
    
    def __add__(self, other):
        if self.shape[0] != other.shape[0] or self.shape[1] != other.shape[1]:
            print("CSR addition shape does not match")
            return
        
        shape = deepcopy(self.shape)
        
        new_data = []
        new_indptr = []
        new_indices = []
        new_indptr.append(0)
        ele_num = 0
        for row in range(shape[0]):
            self_start = self.indptr[row]
            self_end = self.indptr[row+1]
            other_start = other.indptr[row]
            other_end = other.indptr[row+1]
            
            self_ele = self_end-self_start
            other_ele = other_end-other_start
            s_p = self_start
            o_p = other_start
            
            while s_p < self_end or o_p < other_end:
                if s_p == self_end:
                    for i in range(other_end - o_p):
                        new_data.append(other.data[o_p + i])
                        new_indices.append(other.indices[o_p + i])
                    ele_num += other_end - o_p
                    break
                if o_p == other_end:
                    for i in range(self_end - s_p):
                        new_data.append(self.data[s_p + i])
                        new_indices.append(self.indices[s_p + i])
                    ele_num += self_end - s_p
                    break 
                if self.indices[s_p] < other.indices[o_p]:
                    new_data.append(self.data[s_p])
                    new_indices.append(self.indices[s_p])
                    s_p += 1
                elif self.indices[s_p] > other.indices[o_p]:
                    new_data.append(other.data[o_p])
                    new_indices.append(other.indices[o_p])
                    o_p += 1
                else:
                    new_data.append(self.data[s_p] + other.data[o_p])
                    new_indices.append(self.indices[s_p])
                    o_p += 1
                    s_p += 1
                ele_num += 1
            new_indptr.append(ele_num)
            
        return CSR((new_data,new_indices,new_indptr),shape)
    
    def __sub__(self, other):
        if self.shape[0] != other.shape[0] or self.shape[1] != other.shape[1]:
            print("CSR addition shape does not match")
            return
        
        shape = deepcopy(self.shape)
        
        new_data = []
        new_indptr = []
        new_indices = []
        new_indptr.append(0)
        ele_num = 0
        for row in range(shape[0]):
            self_start = self.indptr[row]
            self_end = self.indptr[row+1]
            other_start = other.indptr[row]
            other_end = other.indptr[row+1]
            
            self_ele = self_end-self_start
            other_ele = other_end-other_start
            s_p = self_start
            o_p = other_start
            
            while s_p < self_end or o_p < other_end:
                if s_p == self_end:
                    for i in range(other_end - o_p):
                        new_data.append(-other.data[o_p + i])
                        new_indices.append(other.indices[o_p + i])
                    ele_num += other_end - o_p
                    break
                if o_p == other_end:
                    for i in range(self_end - s_p):
                        new_data.append(self.data[s_p + i])
                        new_indices.append(self.indices[s_p + i])
                    ele_num += self_end - s_p
                    break 
                if self.indices[s_p] < other.indices[o_p]:
                    new_data.append(self.data[s_p])
                    new_indices.append(self.indices[s_p])
                    s_p += 1
                elif self.indices[s_p] > other.indices[o_p]:
                    new_data.append(-other.data[o_p])
                    new_indices.append(other.indices[o_p])
                    o_p += 1
                else:
                    new_data.append(self.data[s_p] - other.data[o_p])
                    new_indices.append(self.indices[s_p])
                    o_p += 1
                    s_p += 1
                ele_num += 1
            new_indptr.append(ele_num)
        return CSR((new_data,new_indices,new_indptr),shape)
    
    def __neg__(self):
        d = []
        for e in self.data:
            d.append(-e)
        return CSR((d,self.indices,self.indptr),self.shape)
    
    def __getitem__(self,tup):
        if type(tup) is tuple:
            row , col = tup
            if row == -1:
                row = self.shape[0] - 1
            if col == -1:
                col = self.shape[1] - 1

            start = self.indptr[row]
            end = self.indptr[row+1]
            for i in range(start,end):
                if self.indices[i] == col:
                    return self.data[i]
            return 0
        
        row = tup
        if row == -1:
            row = self.shape[0]-1
        
        if row > self.shape[0]:
            print("get item row overflow")
            return
        
        shape = (1,self.shape[1])
        data = []
        indptr = []
        indices = []
        indptr.append(0)
        
        start = self.indptr[row]
        end = self.indptr[row+1]
        for i in range(start,end):
            data.append(self.data[i])
            indices.append(self.indices[i])
        indptr.append(end-start)
        return CSR((data,indices,indptr),shape)
    
    def __setitem__(self,tup,value):
        if tup is tuple:
            if len(tup) == 2:
                row , col = tup
                if row == -1:
                    row = self.shape[0] - 1
                if col == -1:
                    col = self.shape[1] - 1
                start = self.indptr[row]
                end = self.indptr[row+1]
                for i in range(start,end):
                    if self.indices[i] == col:
                        self.data[i] = value
                        return
        #print("set item failed")
        
    def to_matrix(self):
        row_data = deepcopy(self.data)
        ind = []
        for i in range(self.shape[0]):
            start = self.indptr[i]
            end = self.indptr[i+1]
            num = end - start
            for j in range(num):
                ind.append([i,self.indices[start+j]])
        return (row_data,ind)
    
    def toarray(self):
        data, ind = self.to_matrix()
        M = np.zeros(self.shape, dtype = object)
        for i in range(len(data)):
            M[ind[i][0],ind[i][1]] = data[i]
        return M
    
    def show(self):
        print("shape: (%d,%d)"%(self.shape[0],self.shape[1]))
        row_data ,ind = self.to_matrix()
        for i in range(len(row_data)):
            print(ind[i],row_data[i])
    
    def renew_nnz(self):
        num = 0
        for e in self.data:
            if e != 0:
                num += 1
        self.nnz = num
    
    def eliminate_zeros(self):
        n = len(self.data)
        p = n-1
        indptr = []
        indptr.append(0)
        row_ele = np.zeros(n,dtype=int)
        current_row = self.shape[0] - 1
        
        for i in range(n):
            while self.indptr[current_row] > p:
                current_row -= 1
            
            if self.data[p] == 0:
                self.data = np.delete(self.data,[p],None)
                self.indices = np.delete(self.indices,[p],None)
            else:
                row_ele[current_row] += 1
            p -= 1
         
        s = 0
        for i in range(n):
            s += row_ele[i]
            indptr.append(s)
        self.indptr = indptr
        self.nnz = len(self.data)
        
    def __rmul__(self,other):
        data = deepcopy(self.data)
        data = other*data
        indptr = deepcopy(self.indptr)
        indices = deepcopy(self.indices)
        shape = deepcopy(self.shape)
        return CSR((data,indices,indptr),shape)
    
    def __mul__(self,other):
        data = deepcopy(self.data)
        data = data*other
        indptr = deepcopy(self.indptr)
        indices = deepcopy(self.indices)
        shape = deepcopy(self.shape)
        return CSR((data,indices,indptr),shape)
    
    def __div__(self,other):
        data = deepcopy(self.data)
        data = data/other
        indptr = deepcopy(self.indptr)
        indices = deepcopy(self.indices)
        shape = deepcopy(self.shape)
        return CSR((data,indices,indptr),shape)
    
    def tocsr(self):
        return self
    
    def vstack(self,other):
        if self.shape[1] != other.shape[1]:
            print("vstack shape does not match")
            return
        
        data = np.hstack((self.data,other.data))
        indices = np.hstack((self.indices,other.indices))
        indptr = deepcopy(self.indptr)
        indptr_add = np.delete(other.indptr,0,None) + indptr[-1]
        indptr = np.hstack((indptr,indptr_add))
        
        shape = (self.shape[0]+other.shape[0], self.shape[1])
        return CSR((data,indices,indptr),shape)
        
        

In [49]:
def matC_sparse(polynomials, mdeg,nvar=2, Float = False ):
    row = []
    col =[]
    ele = []
    
    rows = len(polynomials)
    cols = mdeg**nvar
    
    for i in range(rows):
        for monomial in polynomials[i]:
            s = cols-1
            for j in range(nvar):
                s -= monomial[j+1]*mdeg**(nvar-1-j)
            row.append(i)
            col.append(s)
            if Float:
                ele.append(1.0*monomial[0])
            else:
                ele.append(Fraction(1,1)*monomial[0])
    #M = sp.sparse.csr_matrix((ele, (row, col)), shape=(rows, cols))
    M1 = CSR((ele, (row, col)), shape=(rows, cols))
    return M1
    # return M,M1


In [24]:
a = np.array([1,2,3,4,5])
a = np.delete(a,1,None)
a/Fraction(2,1)

array([Fraction(1, 2), Fraction(3, 2), Fraction(2, 1), Fraction(5, 2)], dtype=object)

# Gaussian Elimination

In [25]:
def GEPP_sparse(C):    
    M1 = C.copy()
    M1 = M1.tocsc()
    n = M1.shape[0]  ## number of rows
    m = M1.shape[1]  ## number of columns

    currentrow = 0
    for i in range(m):
        if M1[currentrow:,i].nnz==0:
            pass  ## do nothing if we have a column of zeros
        else:
            if currentrow < (n-1):
                    #print(i)
                    ## first find index of (abs) maximum value
                non_zero_row = M1[currentrow:n,i]
                sub_index = np.argmax(abs(sp.sparse.find(non_zero_row)[2]))
                index = sp.sparse.find(non_zero_row)[0][sub_index]+currentrow  #### n\n-1 
                    #print(M[currentrow:n,i])
                    #print(index)
                    #print("index is", index,"\n")
                # swap rows
                a_idx = np.where(M1.indices == index)
                b_idx = np.where(M1.indices == currentrow)
                M1.indices[a_idx] = currentrow
                M1.indices[b_idx] = index    
                          ####
                #print("the matrix is now \n",M,"\n")
                #print(M)

                    ## running the elimination step
                for j in range(currentrow+1,n):
                    if M1[j,i]!=0:
                            #print("currentrow column entry is", M[currentrow,i],"\n")
                        multiplier = M1[j,i]/M1[currentrow,i]
                            #print("multiplier is", multiplier,"\n")
                        M1[j,i:] = M1[j,i:] - multiplier*M1[currentrow,i:]
                    else:
                        pass
                    #print("the matrix is after elimination \n",M,"\n")

                    ## incrementing row so the algorithm stops when we have a triangular system
    
                currentrow = currentrow + 1
    
    M1=M1.tocsr()
    
    for j in range(n):
        if M1[j,:].nnz==0:
            continue  ## do nothing if we have a row of zeros
        else:
       

            indptr = M1.indptr
            a = indptr[j]
            b = indptr[j+1]
            M1.data[a:b] = M1.data[a:b]/M1.data[a] 
      

    return M1.tocsr()


# Buchberger's algorithm for multi-variable

In [26]:
## sparse

# input the start th leading
def leading_sparse(row,start=0):
    if row.nnz==0:
        return -1
    if start >= row.nnz:
        return -1
        
        print("leading term overflow")
        print(row)
        print(start)
        sys.exit()
    
    
    return row.indices[start]
    # return sp.sparse.find(row)[1][start]
    

def leading_term_sparse(arg,mdeg,nvar):
    # return leading term without coefficient
    
    result = np.zeros(nvar)
    a = arg
    b = 0
    for i in range(nvar):
        div = mdeg**(nvar-1-i)
        b = a - a/div*div
        a = a/div
        result[i]=mdeg-1-a
        a = b
    return result

def check_multiple_sparse(lead_1,lead_2,nvar):
    # check lead_1 is a multiple of lead_2
    for i in range(nvar):
        if lead_1[i]<lead_2[i]:
            return False
    return True

def shift_col_sparse(lcm,deg_i,mdeg,nvar):
    a = lcm[:-1]-deg_i
    shift_column = 0
    for i in range(nvar):
        shift_column += mdeg**(nvar-1-i)*a[i]
    return int(shift_column)

# input sparse matirx
def shift_sparse(lcm,deg_i,mdeg,nvar,row=-1,leading=-1,S=(-1,-1)):
    row = row.tocsr()
    row_copy = deepcopy(row)
    shift_column = shift_col_sparse(lcm,deg_i,mdeg,nvar)
    if leading-shift_column < 0:
        print(S,leading,shift_column)
        row.show()
        print("degree overflow")
        sys.exit()
    row_copy.indices[:] -= shift_column

    return row_copy

def get_lcm_sparse(a,b): 
    return abs(a * b) / fractions.gcd(a,b) if a and b else 0

def check_multiple_sparse(lead_1,lead_2,nvar):
    # check lead_1 is a multiple of lead_2
    for i in range(nvar):
        if lead_1[i]<lead_2[i]:
            return False
    return True

def remainder_sparse(M,new_row,mdeg, nvar):
    M = M.tocsr()
    flag = True
    m = M.shape[0]
    while flag:
        flag = False
        new_lead = leading_sparse(new_row)

        if new_lead==-1:
            break
        new_lead_term = leading_term_sparse(new_lead,mdeg,nvar)
        for i in range(m):
            row_lead = leading_sparse(M[i])
            row_lead_term = leading_term_sparse(row_lead,mdeg,nvar)
            if check_multiple_sparse(new_lead_term,row_lead_term,nvar):
                #print(i)
                lcm = np.hstack((new_lead_term,[1]))
                old_shifted = shift_col_sparse(lcm,row_lead_term,mdeg,nvar)
                coeff = new_row[0,new_lead]
                
                current_row = deepcopy(M[i])
                #print(current_row)
                current_row.indices[:] -= old_shifted
                #print(current_row)
                
                new_row -= coeff * current_row
                flag = True
                break
    return new_row

def Buchberger_sparse(M, mdeg, nvar, S):
    # check leading term
    a = M[S[0]]
    b = M[S[1]]
    
    arg_a = leading_sparse(a)
    arg_b = leading_sparse(b)
    
    if arg_a==-1 or arg_b==-1:
        return 0,M
        
        print("0 row occur")
        print(S)
        print(M)
        sys.exit()
    
    deg_a = leading_term_sparse(arg_a,mdeg,nvar)
    deg_b = leading_term_sparse(arg_b,mdeg,nvar)
    

    lcm = np.zeros(nvar+1)
    for i in range(nvar):
        lcm[i] = max(deg_a[i],deg_b[i])
        
    if(a[0,arg_a]<0):
        a *= -1
        
    if(b[0,arg_b]<0):
        b *= -1

    lcm[nvar] = get_lcm_sparse(a[0,arg_a],b[0,arg_b])
    

    a_shifted = shift_sparse(lcm,deg_a,mdeg,nvar,a,arg_a,S)
    b_shifted = shift_sparse(lcm,deg_b,mdeg,nvar,b,arg_b,S)
    
    """
    a_shifted.show()
    b_shifted.show()
    print(a[0,arg_a], b[0,arg_b])
    """
    """   
    if a[0,arg_a]!=1 or b[0,arg_b]!=1:
        print("Normalization in Gaussian Elimination Failed")
        print(M)
        sys.exit()
    """
    if a[0,arg_a]> b[0,arg_b]:
        new_row = a_shifted - (a[0,arg_a]/ b[0,arg_b])*b_shifted
    else:
        new_row = (b[0,arg_b]/a[0,arg_a])*a_shifted - b_shifted
    
    
    new_row = remainder_sparse(M,new_row,mdeg, nvar)

    
    if new_row.nnz == 0:
        return 0,M
    
    # normalize new column
    arg_c = leading_sparse(new_row)
    
    if new_row[0,arg_c]!=1:
        new_row.data[:] /= new_row[0,arg_c]
    M = M.vstack(new_row)
    return 1,M




In [27]:
indptr = [0,1,2,4]
indices = [0,1,0,2]
data = [2,3,4,5]
ptr = [0,2,3,5]
dic = [0,2,1,0,2]
d = [1,2,3,4,5]
A =CSR((data, indices, indptr), shape=(3, 3))
B = CSR((d, dic, ptr), shape=(3, 3))
print(A.toarray())
print(B.toarray())
print((A[2]).nnz)
leading_sparse(A[2])

[[2 0 0]
 [0 3 0]
 [4 0 5]]
[[1 0 2]
 [0 3 0]
 [4 0 5]]
2


0

# Reduced 

In [28]:
def find_largest(lead_v):
    # l is the largest
    l = -1
    arg = -1
    
    m = lead_v.shape[0]
    
    for row in range(m):
        if l == -1:
            l = lead_v[row,-1]
            arg = row
            continue
        if lead_v[row,-1] != -1 and lead_v[row,-1]<l:
            l = lead_v[row,-1]
            arg = row
    if l == -1:
        return -1
    return arg

def renew_lead_v(M,mdeg,nvar,lead_v,row,starts,tol=10**-4):
    m = M.shape[0]
    
    lead_v[row,-1] = leading_sparse(M[row],starts[row])
    while abs(M[row,lead_v[row,-1]]) < tol:
        
        M[row,lead_v[row,-1]]=0
        M.eliminate_zeros()
        lead_v[row,-1] = leading_sparse(M[row],starts[row])

        if lead_v[row,-1] == -1:
            return M,lead_v    
    if lead_v[row,-1] == -1:
        return M,lead_v   
    monomial = leading_term_sparse(lead_v[row,-1],mdeg,nvar)
    for j in range(nvar):
        lead_v[row,j] = monomial[j]
    return M,lead_v
    
def reduced_form(M,mdeg,nvar,tol):
    m,n = M.shape
    
    # the vector store the leading term
    lead_v = np.zeros((m,nvar+1),dtype = int)
    
    # vector stores the i the leading term
    starts = np.zeros(m,dtype = int)
    
    # initialization leading_vector
    for row in range(m):
        M,lead_v = renew_lead_v(M,mdeg,nvar,lead_v,row,starts,tol)
    
    large_row = find_largest(lead_v)
    while large_row != -1:
        flag = False
        for row in range(m):
            if row != large_row and starts[row] == 0:
                lead_arg = leading_sparse(M[row])
                lead_term = leading_term_sparse(lead_arg,mdeg,nvar)
                if check_multiple_sparse(lead_v[large_row,:-1],lead_term,nvar):

                    shift_col = shift_col_sparse(lead_v[large_row],lead_term,mdeg,nvar)
                    indices = deepcopy(M[row].indices) - shift_col
                    indptr = np.zeros(m+1,dtype = int)
                    for i in range(large_row+1,m+1):
                        indptr[i] = indices.shape[0]
                    data = deepcopy(M[row].data)
                    M_subtrac = sp.sparse.csr_matrix((data, indices, indptr), shape=(m, n))
                    c = M[large_row,lead_v[large_row,-1]]/M[row,lead_v[row,-1]]
                    M -= c*M_subtrac
                    flag = True
                    break
        if flag == False:
            starts[large_row]+=1

        M,lead_v=renew_lead_v(M,mdeg,nvar,lead_v,large_row,starts,tol)
        large_row = find_largest(lead_v)
    
    return M
    #print_all(matrix_to_array_sparse(M,mdeg,nvar),['A0','x0','A1','x1'],nvar)

    

# Combine them together

In [29]:
def delete_row_csr(mat, i):
    if not isinstance(mat, sp.sparse.csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    n = mat.indptr[i+1] - mat.indptr[i]
    if n > 0:
        mat.data[mat.indptr[i]:-n] = mat.data[mat.indptr[i+1]:]
        mat.data = mat.data[:-n]
        mat.indices[mat.indptr[i]:-n] = mat.indices[mat.indptr[i+1]:]
        mat.indices = mat.indices[:-n]
    mat.indptr[i:-1] = mat.indptr[i+1:]
    mat.indptr[i:] -= n
    mat.indptr = mat.indptr[:-1]
    mat._shape = (mat._shape[0]-1, mat._shape[1])

def trun_zero_sparse(M):
    while M[-1,:].nnz==0:
        M = M[:-1,:]
    return M

def normalize_leading_sparse(M):
    m = M.shape[0]
    for i in range(m):
        lead = leading_sparse(M[i])
        if lead == -1:
            #delete_row_csr(M, i)
            continue
        indptr = M.indptr
        a = indptr[i]
        b = indptr[i+1]
        M.data[a:b] = M.data[a:b]/M.data[a]
    return M

In [30]:
def check_nvar(row):
    ele = 0
    arg = -1
    nvar = row.shape[0]
    for i in range(nvar):
        if row[i] == 1:
            ele += 1
            arg = i
    if ele == 1:
        return arg
    if ele == 0:
        return -2
    return -1

def check_terminate(M,mdeg,nvar):
    M.eliminate_zeros()
    m = M.shape[0]
    
    Var = np.zeros((m,nvar))
    for i in range(m):
        for j in range(M[i].nnz):
            lead = leading_sparse(M[i],j)
            if lead == -1:
                print("check_terminate problem")
                sys.exit()
            term = leading_term_sparse(lead,mdeg,nvar)
            for k in range(nvar):
                if term[k] != 0:
                    Var[i,k] = 1.
    C_E = np.zeros(nvar,dtype = int)
    flag = True
    
    #print(Var)
    while(flag):
        flag = False
        row_num = 0
        while row_num<Var.shape[0]:
            c = check_nvar(Var[row_num])
            #print(row_num,c)
            #print(Var)
            if c > -1:
                C_E[c] = 1
                Var[:,c] = 0
                Var=np.delete(Var, row_num, 0)
                flag = True
                break
            if c == -2:
                Var=np.delete(Var, row_num, 0)
                continue
            row_num += 1
    #print(C_E)
    if sum(C_E) == nvar:
        return True
    return False

In [31]:
# details is for debug or process, combined with symbols
def run_sparse(M, mdeg, nvar,tol = 10**-14,details = False,cal = False,Gro = False,Red = True,symbols = []):
    #M = GEPP_sparse(M)
    
    max_row = M.shape[0]-1
    current_row = 0
    flag = -1
    M=reduced_form(M,mdeg,nvar,tol)
    if details:
        print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,symbols,cal)
        print("")
    #M=normalize_leading_sparse(M)
    while current_row != max_row:
        #print(current_row,max_row)
        current_row +=1
        #M = GEPP(M)
        for i in range(current_row):
            M=normalize_leading_sparse(M)
            flag,M = Buchberger_sparse(M, mdeg, nvar, [current_row,i])
            if flag == 1 and Red:
                M=reduced_form(M,mdeg,nvar,tol)
                if details:
                    print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,symbols,cal)
                    print("")
                if Gro == False:
                    if check_terminate(M,mdeg,nvar):
                        M=normalize_leading_sparse(M)
                        return M
        #time.sleep(1)
        #M = GEPP(M)
        #M = trun_zero(M)
        M=normalize_leading_sparse(M)
        #print(M)
        max_row = M.shape[0]-1
    M=normalize_leading_sparse(M)
    return M


In [32]:
def matrix_to_array_sparse(M,mdeg,nvar):
    array = []
    m = M.shape[0]
    for i in range(m):
        array.append([])
        index = 0
        nnz = M[i].nnz
        for j in range(nnz):
            index = leading_sparse(M[i],j)
            L_T = leading_term_sparse(index,mdeg,nvar)
            add = [M[i,index]]
            add.extend(L_T)
            array[i].append(add)

    return array

def print_array_sparse(array):
    for p in array:
        print(p)
        

# Testing

In [33]:
p1 = [(1,2,0)]

p2 = [(1,1,1),(1,0,2)]
p4 = [(1,2,0),(1,1,1),(1,0,2)]
polynomials = [p1,p2,p4]
mdeg = 6
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar,Float=True)
M=reduced_form(M,mdeg,nvar,tol = 10**-5)
print_all(matrix_to_array_sparse(M,mdeg,nvar),2,['x','y']) 
M=normalize_leading_sparse(M)
#print_all(matrix_to_array_sparse(M,mdeg,nvar),2,['x','y']) 
flag,M = Buchberger_sparse(M, mdeg, nvar, [2,1])
print_all(matrix_to_array_sparse(M,mdeg,nvar),2,['x','y']) 


xy+y^2\\
x^2+xy+y^2\\

xy+y^2\\
x^2+xy+y^2\\
y^3\\



Here we test $p_1=x^2$ and $p_2=xy+y^2$. And get Grobner basis $g_1=x^2, g_2=xy+y^2, g_3 = y^3$.

In [34]:
p1 = [(1,2,0)]
p2 = [(1,1,1),(1,0,2)]
polynomials = [p1,p2]
mdeg = 5
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar,Float=True)
#print(M)
M = run_sparse(M, mdeg,nvar,Gro=True,Red=False)
#M.toarray()
print_all(matrix_to_array_sparse(M,mdeg,nvar),2,['x','y']) 


x^2\\
xy+y^2\\
y^3\\



In [35]:
%%timeit
p1 = [(1,2,0)]
p2 = [(1,1,1),(1,0,2)]
polynomials = [p1,p2]
mdeg = 5
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,Gro=True)
#M.toarray()



100 loops, best of 3: 2.55 ms per loop


Here we test $p_1=xyz+xy$ and $p_2=x^2y^2z+yz$. And get Grobner basis $g_1=x^2y^2z+yz, g_2=xyz+xy, g_3=x^2y^2-2yz, g_4 = yz^2+yz$.

In [60]:
p1 = [(1,1,1,1),(1,1,1,0)]
p2 = [(2,0,1,1),(1,2,2,1)]
polynomials = [p1,p2]
mdeg = 5
nvar = 3
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,Gro=True,Red=False,details = True, symbols = ['x','y','z'])
#M.toarray()
print_all(matrix_to_array_sparse(M,mdeg,nvar),3,['x','y','z']) 


xyz+xy\\
-x^2y^2+2.00yz\\


xyz+xy\\
x^2y^2-2.00yz\\
yz^2+yz\\



# Solving Gaussian Quadrature for $n=2$

In [51]:
p1= [(1,1,0,0,0),(1,0,0,1,0),(-2,0,0,0,0)]
p2= [(1,1,1,0,0),(1,0,0,1,1)]
p3= [(1,1,2,0,0),(1,0,0,1,2),(-2.0/3,0,0,0,0)]
p4= [(1,1,3,0,0),(1,0,0,1,3)]

p1= [(1,1,0,0,0),(1,0,0,1,0),(-2,0,0,0,0)]
p2= [(1,1,1,0,0),(1,0,0,1,1)]
p3= [(1,1,2,0,0),(1,0,0,1,2),(Fraction(-2,3),0,0,0,0)]
p4= [(1,1,3,0,0),(1,0,0,1,3)]
polynomials = [p1,p2,p3,p4]
mdeg = 10
nvar = 4
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
M.show()
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['A0','x0','A1','x1'])

shape: (5,10000)
[0, 8999] 1
[0, 9999] -1
[2, 9997] 1
[2, 9999] -1/3
[3, 9899] 1
[3, 9998] 1
[4, 9989] 1
[4, 9997] 3
[4, 9999] -2
A0-1.00\\
x1^2-0.33\\
x0+x1\\
A1+3.00x1^2-2.00\\



In [ ]:
%memit matC_sparse(polynomials,mdeg,nvar)


In [ ]:
M = matC_sparse(polynomials,mdeg,nvar)

def T(M,mdeg,nvar,loop=50):
    for i in range(loop):
        M = matC_sparse(polynomials,mdeg,nvar)
        run_sparse(M, mdeg,nvar)
%memit run_sparse(M, mdeg,nvar)


In [ ]:
M = matC_sparse(polynomials,mdeg,nvar)
%timeit run_sparse(M, mdeg,nvar) 


# Symmetric Gaussian Quadrature

## n = 3

In [43]:
n = 3
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar,Float = True)
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = True,cal=True,Gro=True,symbols=GQ_sym(n))
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 

print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,True),cal = False,precision = True)

2.0*w0-1.11111111111,
0.888888888889*x1,
-0.6*w1+0.533333333333,
0.666666666667*x0**2-0.4,


w_0-0.555555555556\\
x_1\\
w_1-0.888888888889\\
x_0^2-0.6\\



In [ ]:
%%timeit
n = 3
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()

In [ ]:
%%memit
n = 3
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

## n = 4

In [ ]:
n = 4
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=True),cal = False, precision = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = True,cal=True,symbols=GQ_sym(n))
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 

#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)
M = normalize_leading_sparse(M)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=True),cal = False,precision= True)

In [ ]:
%%timeit
n = 4
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

In [ ]:
%%memit
n = 4
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)


In [ ]:
A0=0.6521451548625461
x0=-0.3399810435848563
A1=0.3478548451374538
x1=0.8611363115940526
print(A0+A1-1.0,
x0**2.0*A1+0.6*A1+x1**2.0-0.990476190476,
-0.444444444444*x1**4.0+0.380952380952*x1**2.0-0.0380952380952,
-0.2*x0**2.0-0.333333333333*x1**4.0+0.0857142857143*x1**2.0+0.142857142857,
0.190476190476*A1*x1**2.0+0.114285714286*A1-0.0888888888889)

# n = 5

In [ ]:
n = 5
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=True),cal = False, precision = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = True,cal=True,symbols=GQ_sym(n))
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)
M = normalize_leading_sparse(M)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=True),cal = False,precision= True)

In [ ]:
%%timeit
n = 5
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

In [ ]:
%%memit
n = 5
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

In [ ]:
A2=0.5688888888888889
x2=0.0000000000000000
A1=0.4786286704993665
x1=-0.5384693101056831
A0=0.2369268850561891
x0=-0.9061798459386640
print(A0+A1-0.715555555556,
0.568888888889*x2,
x0**2.0*A1+0.230769230769*A1+0.715555555556*x1**2.0-0.710959164292,
0.238095238095*A2-0.13544973545,
0.106666666667*x1**4.0-0.118518518519*x1**2.0+0.0253968253968,
0.285714285714*x0**2.0+0.4*x1**4.0-0.15873015873*x1**2.0-0.222222222222,
-0.256790123457*A1*x1**2.0-0.0592592592593*A1-0.2*A2+0.177777777778)

In [ ]:
%%timeit
n = 1
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

In [ ]:
plt.ylabel('ms')
plt.xlabel('number of points')
plt.title('Time cost of Guassian Qudrature')
plt.axis([1, 6,0,150])
plt.plot([1,2,3,4,5],[3.8,7.88,34.1,49.4,101])

# Solving Gaussian Quadrature without symmetry

In [ ]:
n = 2
polynomials = [[[1, 1, 0, 0, 0], [1, 0, 0, 1, 0], [-2.0, 0.0, 0.0, 0.0, 0.0]], [[1, 1, 1, 0, 0], [1, 0, 0, 1, 1]], [[1, 1, 2, 0, 0], [1, 0, 0, 1, 2], [Fraction(-2,3), 0.0, 0.0, 0.0, 0.0]], [[1, 1, 3, 0, 0], [1, 0, 0, 1, 3]]]
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=False),cal = False,precision = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=True),cal = False,precision = True)

## n = 2

In [61]:
n = 2
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=False),cal = False,precision = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=True),cal = True,precision = True)
#print(M)

w0+w1-2\\
w0x0+w1x1\\
w0x0^2+w1x1^2-2/3\\
w0x0^3+w1x1^3\\

w_0-1,
x_1**2-1/3,
x_0+x_1,
w_1+3*x_1**2-2,



In [ ]:
%%timeit
n = 2
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=True),cal = False,precision = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)


In [ ]:
%%memit
n = 2
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n,Latex=True),cal = False,precision = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)

## n = 3

In [ ]:
Fraction(2,9)+Fraction(1,9)*2.0

In [ ]:
n = 3
polynomials = [[[1, 1, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0], [-2, 0, 0, 0, 0, 0, 0]], [[1, 1, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, 0], [1, 0, 0, 0, 0, 1, 1]], [[1, 1, 2, 0, 0, 0, 0], [1, 0, 0, 1, 2, 0, 0], [1, 0, 0, 0, 0, 1, 2], [Fraction(-2,3), 0, 0, 0, 0, 0, 0]], [[1, 1, 3, 0, 0, 0, 0], [1, 0, 0, 1, 3, 0, 0], [1, 0, 0, 0, 0, 1, 3]], [[1, 1, 4, 0, 0, 0, 0], [1, 0, 0, 1, 4, 0, 0], [1, 0, 0, 0, 0, 1, 4], [Fraction(-2,5), 0, 0, 0, 0, 0, 0]], [[1, 1, 5, 0, 0, 0, 0], [1, 0, 0, 1, 5, 0, 0], [1, 0, 0, 0, 0, 1, 5]]]
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)

In [64]:
n = 3
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar))
M = 1. *M
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)


w0+w1-0.555555555556*x2**2-1.11111111111,
w1*x2+2.0*x1*w2+0.555555555556*x1*x2**2-1.77777777778*x1-0.888888888889*x2,
x2**3-0.6*x2,
x1**2+x1*x2+x2**2-0.6,
x0+x1+1.66666666667*x2**3,
w1*w2-1.87654320988*w1-2.96296296296*x1**2+1.11111111111*x1*w2**2*x2-6.2962962963*x1*w2*x2+0.925925925926*x1*x2+w2**2-1.11796982167*x2**2+1.53635116598,
w1*x1+0.5*w1*x2+1.0625*x1*w2+0.3125*x1*x2**2-1.5*x1-0.555555555556*x2**3-0.111111111111*x2,
w2+0.555555555556*x2**2-0.888888888889,



In [65]:
w0=0.8888888888888888
x0=0.0000000000000000
w1=0.5555555555555556
x1=-0.7745966692414834
w2=0.5555555555555556
x2=0.7745966692414834

print(w0+w1-0.555555555556*x2**2-1.11111111111,
w1*x2+2.0*x1*w2+0.555555555556*x1*x2**2-1.77777777778*x1-0.888888888889*x2,
x2**3-0.6*x2,
x1**2+x1*x2+x2**2-0.6,
x0+x1+1.66666666667*x2**3,
w1*w2-1.87654320988*w1-2.96296296296*x1**2+1.11111111111*x1*w2**2*x2-6.2962962963*x1*w2*x2+0.925925925926*x1*x2+w2**2-1.11796982167*x2**2+1.53635116598,
w1*x1+0.5*w1*x2+1.0625*x1*w2+0.3125*x1*x2**2-1.5*x1-0.555555555556*x2**3-0.111111111111*x2,
w2+0.555555555556*x2**2-0.888888888889)

8.44213587925e-13 1.42852396579e-12 5.55111512313e-17 1.11022302463e-16 1.54920520856e-12 2.57327492648e-12 -1.20625731626e-13 1.55653268052e-13


In [ ]:
%%memit
n = 3
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

In [ ]:
%%timeit
n = 3
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

In [67]:
n = 4
polynomials = GQ(n)
mdeg = 30
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar))
#M = 1. *M
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)

w0+w1+w2+w3-2,
w1**2*x1*x2**2*w3-2*w1**2*x1*x2**2+2*w1**2*x1*x2*w3*x3-2*w1**2*x1*x2*x3-3*w1**2*x1*w3*x3**2+4/3*w1**2*x1-w1**2*x2**2*w3*x3-2*w1**2*x2*w3*x3**2+2/3*w1**2*x2+3*w1**2*w3*x3**3-2/3*w1**2*x3-16/9*w1*x1*w2*w3*x3**2+2/9*w1*x1*w2*x3**2+2/3*w1*x1*w2-4/9*w1*x1*x2*w3**2*x3+10/9*w1*x1*x2*w3*x3-4/9*w1*x1*x2*x3-4/3*w1*x1*w3**2*x3**2+20/9*w1*x1*w3*x3**2+2/9*w1*x1*w3-4/9*w1*x1*x3**2+20/9*w1*x1+4/3*w1*w2*x2**2*w3*x3-2/3*w1*w2*x2**2*x3-22/9*w1*w2*x2*w3*x3**2-2/7*w1*w2*x2*w3-4/9*w1*w2*x2*x3**2+40/21*w1*w2*x2+2/9*w1*w2*w3*x3**3+2/3*w1*w2*w3*x3+2/9*w1*w2*x3**3-4/21*w1*w2*x3+1/3*w1*x2**3*w3**2-4/3*w1*x2**3*w3+4/3*w1*x2**3-1/9*w1*x2**2*w3**2*x3-2/9*w1*x2**2*w3*x3+8/9*w1*x2**2*x3-1/9*w1*x2*w3**2*x3**2-2/7*w1*x2*w3**2+16/9*w1*x2*w3*x3**2+38/21*w1*x2*w3+8/9*w1*x2*x3**2-8/7*w1*x2-w1*w3**2*x3**3+2/3*w1*w3**2*x3+14/9*w1*w3*x3**3-218/189*w1*w3*x3-4/9*w1*x3**3+268/189*w1*x3+1/3*x1*w2*x2**2*w3**2-4/3*x1*w2*x2**2*w3+4/3*x1*w2*x2**2-2/3*x1*w2*x2*w3**2*x3+4/3*x1*w2*x2*w3*x3+1/3*x1*w2*w3**2*x3**2-2/3*x1*w2

In [85]:
print_all(matrix_to_array_sparse(M[11],mdeg,nvar),nvar,GQ_sym(n),cal = True)

w1*x1*w2*x2-1375/162*w1*x1*w2*w3*x3**3+17479187/438912*w1*x1*w2*x3**3+137/27*w1*x1*w2*x3+441350/1143*w1*x1*x2**2*x3**3-14119/381*w1*x1*x2**2*x3+30485033/329184*w1*x1*x2*w3**2*x3**2-127609/12192*w1*x1*x2*w3**2+189081631/2633472*w1*x1*x2*w3*x3**2-532609/97536*w1*x1*x2*w3+36779573/438912*w1*x1*x2*x3**2-1986311/146304*w1*x1*x2-11093011/109728*w1*x1*w3**2*x3**3+127609/12192*w1*x1*w3**2*x3-304786541/2633472*w1*x1*w3*x3**3+41652115/2633472*w1*x1*w3*x3-11222021/109728*w1*x1*x3**3+17909/5184*w1*x1*x3+220675/381*w1*w2*x2**4*x3**2-14119/254*w1*w2*x2**4+2681832685/585216*w1*w2*x2**3*x3**3-283387811/585216*w1*w2*x2**3*x3+111981/254*w1*w2*x2**2*w3**2*x3**2-6305/127*w1*w2*x2**2*w3**2+294356551/109728*w1*w2*x2**2*w3*x3**2-2513955/8128*w1*w2*x2**2*w3+2943393913/1755648*w1*w2*x2**2*x3**2-38126563/195072*w1*w2*x2**2-175031/508*w1*w2*x2*w3**2*x3**3+18661/508*w1*w2*x2*w3**2*x3+13857604/10287*w1*w2*x2*w3*x3**3-85340515/512064*w1*w2*x2*w3*x3+584847863/1755648*w1*w2*x2*x3**3-402895711/12289536*w1*w2*x2*x3-247

In [76]:
x3 = 0.8611363115940526
x3**4-6./7*x3**2+3./35

8.326672684688674e-17

# Random polynomial

In [ ]:
def ran(bound):
    a = int(np.random.random()*bound)
    return a

def Random_P(nvar = 2,ma = 2, m = 2, n =2 , coe = 4):
    Polys = []
    for i in range(m):
        poly = []
        for j in range(n):
            term = []
            term.append(1.*ran(coe)+1)
            for k in range(nvar):
                term.append(1.*ran(ma))
            poly.append(term)
        Polys.append(poly)
    return Polys
            

In [ ]:
n = 3
ma = 3
nvar = 3
m=3
mdeg = 100
coe = 4
polynomials = Random_P(nvar,ma, m , n  , coe)
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar)

In [ ]:
n = 3
ma = 3
nvar = 3
m=3
mdeg = 100
coe = 4
polynomials = Random_P(nvar,ma, m , n  , 1)
print(polynomials)
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'])
#print(M)
print("")
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal =True)

In [ ]:
polynomials =[[[1.0, 2.0, 2.0, 2.0], [1.0, 0.0, 1.0, 2.0], [1.0, 2.0, 2.0, 1.0]], [[1.0, 1.0, 2.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 0.0]], [[1.0, 1.0, 0.0, 0.0], [1.0, 1.0, 2.0, 0.0], [1.0, 2.0, 2.0, 0.0]]]
n = 3
ma = 3
nvar = 3
m=3
mdeg = 100
coe = 4
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal = False)
#print(M)
print("")
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=False)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal =False)

In [ ]:
polynomials = [[[1.0, 2.0, 0.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.0]],
               [[1.0, 2.0, 1.0, 0.0], [1.0, 1.0, 1.0, 0.0], [1.0, 2.0, 2.0, 2.0]], 
               [[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 0.0, 2.0], [1.0, 0.0, 0.0, 1.0]]
              ]#,[[1.0,0,0.,1.],[-1.,.0,.0,0]]]


In [ ]:
n = 3
ma = 3
nvar = 3
m=3
mdeg = 100
coe = 4
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal = False)
#print(M)
print("")
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True,Gro=False,symbols = ['x','y','z'])
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal =False)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal =True)


In [ ]:
z = 1.
y = -.5+.5*np.sqrt(3)
x = 2.*y**2-1


In [ ]:
print(x**2.0*z+x*y*z+x*y,
x**2.0*y**2.0*z**2.0+x**2.0*y+x*y,
x*y*z+x*z**2.0+z,
z-1.0)

In [ ]:
z = 1.
y = -.5-.5*np.sqrt(3)
x = 2.*y**2-1


In [ ]:
print(x**2.0*z+x*y*z+x*y,
x**2.0*y**2.0*z**2.0+x**2.0*y+x*y,
x*y*z+x*z**2.0+z,
z-1.0)


In [ ]:
z = 0.
y = 10.
x = 0
print(x**2.0*z+x*y*z+x*y,
x**2.0*y**2.0*z**2.0+x**2.0*y+x*y,
x*y*z+x*z**2.0+z)

# Constant Weight

In [ ]:
def CWS(n=2):
    odd = False
    if n-n/2*2 == 1:
        odd = True
    result = []
    w = 2./n
    
    if True:
        for i in range(n+1):
            p = []
            if i == 0:
                continue
            for j in range(n/2):
                p_s = np.zeros(n/2+n - n/2*2+1)
                p_s[0] = w
                p_s[j+1] = i
                p.append(p_s.tolist())
                
                p_s = np.zeros(n/2+n - n/2*2+1)
                p_s[0] = w
                if i-i/2*2 >0:
                    p_s[0] = -w
                p_s[j+1] = i
                p.append(p_s.tolist())
            if odd:
                p_s = np.zeros(n/2+n - n/2*2+1)
                p_s[0] = w
                p_s[n/2+n - n/2*2] = i
                p.append(p_s.tolist())
            
            if i-i/2*2 == 0 :
                p_s = np.zeros(n/2+n - n/2*2+1)
                p_s[0] = -2./(i+1)
                p.append(p_s.tolist())
            
            result.append(p)
        return result


## n = 1

In [ ]:
%%timeit
n = 1
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()

## n = 2

In [ ]:
n = 2
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x_0'],cal = False, precision = True)

In [ ]:
%%timeit
n = 2
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x_0'],cal = False, precision = True)

## n = 3

In [ ]:
n = 3
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x_0','x_1'],cal = False, precision = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x_0','x_1'],cal = False, precision = True)

In [ ]:
%%timeit
n = 3
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()

## n = 4

In [ ]:
n = 4
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x_0','x_1'],cal = False, precision = True)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x_0','x_1'],cal = False, precision = True)

In [ ]:
%%timeit
n = 4
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()

In [ ]:
de = (2./3)**2-4.*2/90
de = np.sqrt(de)
a1 = (2./3+de)/2
a2 = (2./3-de)/2
a1 = np.sqrt(a1)
a2 = np.sqrt(a2)
print(a1,a2)

def fff(aa):
    return 3.0*aa**4-aa**2
c = np.sqrt(.6-fff(a1))
print(c)

## n = 5

In [ ]:
n = 5
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)

In [ ]:
%%timeit
n = 5
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()

In [ ]:
plt.ylabel('ms')
plt.xlabel('number of points')
plt.title('Time cost of Constant Weight')
plt.axis([1, 6,0,40])
plt.plot([1,2,3,4,5],[0.853,1.54,5.98,12.1,27.6])

## Experiment for comparision 

In [ ]:
polynomials = [[[1,1000000,0],[1,0,1000000]],
               [[1,1000000,0]]]
#print_array_sparse(polynomials)
mdeg = 1000002
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = True,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)

In [ ]:
%%timeit
polynomials = [[[1,1000000,0],[1,0,1000000]],
               [[1,1000000,0]]]
#print_array_sparse(polynomials)
mdeg = 1000002
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)


In [ ]:
%%memit
polynomials = [[[1,10000,0],[1,0,10000]],
               [[1,10000,0]]]
#print_array_sparse(polynomials)
mdeg = 1000002
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)

In [ ]:
c = 10**4
polynomials = [[[1.0, 2.0*c, 0.0, c], [1.0,c , c, c], [1.0, c, c, 0.0]],
               [[1.0, 2.0*c, c, 0.0], [1.0, c, c, 0.0], [1.0, 2.0*c, 2.0*c, 2.0*c]], 
               [[1.0, c, c, c], [1.0, c, 0.0, 2.0*c], [1.0, 0.0, 0.0, c]]]

In [ ]:
mdeg = 10*c+2
nvar = 3
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)

In [ ]:
%%timeit
mdeg = 100002
nvar = 3
M=matC_sparse(polynomials,mdeg,nvar)
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)
#print(M)
run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)

In [ ]:
%%memit
mdeg = 200000
nvar = 3
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
#print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)